<h2><center>Data Analytics: Extraction</center></h2>

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# import data
data = pd.read_csv('Analytics Test Data - CSS US&C Analytics Associate - Orders.csv')

In [5]:
# revised the data importation and merge two sheets
orders = pd.read_excel('Analytics Test Data - CSS US&C Analytics Associate.xlsx', sheet_name = 'Orders')
stores = pd.read_excel('Analytics Test Data - CSS US&C Analytics Associate.xlsx', sheet_name = 'Stores', index_col = 'store_id')
data_all = orders.join(stores, on = 'StoreID')

In [4]:
# data preview
data_all.head()

,StoreID,Store,OrderNumber,Subtotal,Delivery Fee,Total,DeliveryPartner,OrderStatus,orderDate,facility_name,brand_name,org_name
0,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04c9b627-8263-45f9-9219-d67cf56d0a13,12.99,1.13,NaN,postmates,Delivered,2020-10-26 17:56:05,Chema's Chinese - Los Angeles,Chema's Chinese,Tasty Co.
1,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04fbb2d8-ae72-4ebb-b808-59ccebb839fe,24.47,2.13,NaN,doordash,Delivered,2020-10-23 01:23:55,Chema's Chinese - Los Angeles,Chema's Chinese,Tasty Co.
2,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,05fe492e-33e1-4360-99fc-427c9f19fc8c,15.46,1.35,NaN,doordash,Picked Up,2020-10-28 20:08:20,Chema's Chinese - Los Angeles,Chema's Chinese,Tasty Co.
3,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,09c0a9a0-a647-42df-9a70-1eb4069292ee,29.95,2.61,NaN,doordash,Delivered,2020-10-27 01:51:59,Chema's Chinese - Los Angeles,Chema's Chinese,Tasty Co.
4,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0c1213dc-82c2-4625-aee0-816bc84a49d2,35.97,3.13,NaN,doordash,Delivered,2020-10-23 02:08:55,Chema's Chinese - Los Angeles,Chema's Chinese,Tasty Co.


In [6]:
# desbribe some data
data_all.describe()

,Subtotal,Delivery Fee,Total
count,4989.000000,4864.000000,2297.00000
mean,28.768360,2.183374,35.06606
std,22.432843,1.843490,26.19978
min,0.000000,0.000000,0.00000
25%,15.950000,1.200000,17.53000
50%,22.440000,1.700000,28.32000
75%,34.500000,2.720000,43.19000
max,656.760000,47.060000,435.78000


In [7]:
# data info
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4989 entries, 0 to 4988
Data columns (total 12 columns):
StoreID            4989 non-null object
Store              4989 non-null object
OrderNumber        4989 non-null object
Subtotal           4989 non-null float64
Delivery Fee       4864 non-null float64
Total              2297 non-null float64
DeliveryPartner    4989 non-null object
OrderStatus        4989 non-null object
orderDate          4989 non-null datetime64[ns]
facility_name      4989 non-null object
brand_name         4989 non-null object
org_name           4989 non-null object
dtypes: datetime64[ns](1), float64(3), object(8)
memory usage: 467.8+ KB


<h3>1. What was the subtotal revenue of the hightest grossing delivery platform, excluding cancellations?</h3>

In [8]:
# find the subtotal revenue of the highest grossing delivery platform
subtotals = data.groupby('DeliveryPartner')['Subtotal'].sum()
print(subtotals)

DeliveryPartner
chownow       11875.12
doordash      65278.84
grubhub        4585.56
grubhubweb    20465.92
postmates      5403.80
ubereats      35916.11
Name: Subtotal, dtype: float64


<h3>2. What was the subtotal revenue for pickup orders for the best performing store on a single delivery partner?</h3>

In [9]:
# select data that include pickup orders
data_pickup = data[data['OrderStatus'] == 'Picked Up']

In [10]:
# find different partners
partners = data['DeliveryPartner'].unique()
print(partners)

['postmates' 'doordash' 'ubereats' 'grubhub' 'grubhubweb' 'chownow']


In [11]:
# find the best performing store on each delivery platform and their subtotal revenue
for partner in partners:
    pickup_partner =  data_pickup[data_pickup['DeliveryPartner'] == partner]
    partner_pickup = pickup_partner.groupby('StoreID')['Subtotal'].sum().max()
    print(partner, partner_pickup)

postmates 357.94
doordash 2352.05
ubereats 336.94000000000005
grubhub 312.9800000000001
grubhubweb 576.2900000000001
chownow 246.45


<h3>3. What was the average subtotal value for delivered orders across all stores in Travis' Thai brand</h3>

In [12]:
# select data that include delivered orders
data_deliver = data[data['OrderStatus'] == 'Delivered']

In [13]:
# select Travis' Thai brand's data
travis_condition = data_deliver.Store.str.startswith('Travis\' Thai')
travis_deliver = data_deliver[travis_condition]
# travis_deliver.head()

In [14]:
# second method to select Travis' Thai brand' data
import re
travis_condition2 = ~data_deliver.Store.str.match(r'^(Travis\' Thai.*)').isnull()
travis_deliver2 = data_deliver[travis_condition2]
# travis_deliver2.head()

In [15]:
# calculatethe average subtotal for delivered orders across Travis' Thai brand
travis_average = travis_deliver.Subtotal.mean()
print(travis_average)

21.968498168498225


<h3>4. What was the total revenue growth rate percentage between the week of 10/11 vs. the week of 10/18 for Gina's Grilled Cheese - Jacksonville?</h3>

In [16]:
# select data for the Gina's Grilled Cheese - Jacksonville
data_ginaJack = data[data['Store'] == 'Gina\'s Grilled Cheese - Jacksonville']

In [17]:
# convert the orderDate column to datetime format
data_ginaJack.loc[:,'orderDate'] = pd.to_datetime(data_ginaJack.loc[:,'orderDate'])

# check if the format of orderDate has changed to datetime
# data_ginaJack.info()

C:\Users\titi1\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
# add a column with only date not the time
data_ginaJack['orderDate_date'] = data_ginaJack.orderDate.dt.date

C:\Users\titi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
from datetime import datetime

# select data in the 10/11 week - method 1
date_1011 = datetime.strptime('2020-10-11', '%Y-%m-%d').date()
date_1017 = datetime.strptime('2020-10-17', '%Y-%m-%d').date()
mask = (data_ginaJack['orderDate_date'] >= date_1011) & (data_ginaJack['orderDate_date'] <= date_1017)
data_ginaJack_1011 = data_ginaJack.loc[mask]
# data_ginaJack_1011.head()

In [20]:
# select data in the 10/11 week - method 2
# set the date to be index
data_ginaJack_indexed = data_ginaJack.sort_values(['orderDate_date']).set_index(['orderDate_date'])

# select the data in the 10/11 week
date_1011 = datetime.strptime('2020-10-11', '%Y-%m-%d').date()
date_1017 = datetime.strptime('2020-10-17', '%Y-%m-%d').date()
data_ginaJack_1011_method2 = data_ginaJack_indexed.loc[date_1011:date_1017]
# data_ginaJack_1011_method2.head()

In [21]:
# calculate the total revenue in the week of 10/11
totalRev_1011 = data_ginaJack_1011['Subtotal'].sum() + data_ginaJack_1011['Delivery Fee'].sum()
print(totalRev_1011)

1474.63


In [22]:
# select data in the 10/18 week
date_1018 = datetime.strptime('2020-10-18', '%Y-%m-%d').date()
date_1025 = datetime.strptime('2020-10-25', '%Y-%m-%d').date()
mask_18 = (data_ginaJack['orderDate_date'] >= date_1018) & (data_ginaJack['orderDate_date'] <= date_1025)
data_ginaJack_1018 = data_ginaJack.loc[mask_18]
# data_ginaJack_1018.head()

In [23]:
# calculate the total revenue in the week of 10/18
totalRev_1018 = data_ginaJack_1018['Subtotal'].sum() + data_ginaJack_1018['Delivery Fee'].sum()
print(totalRev_1018)

4798.69


In [24]:
# calculate the percentage growth of total revenue from the week 0f 10/11 to 10/18
percentage_1011_1018 = (totalRev_1018 - totalRev_1011)/totalRev_1011
print(percentage_1011_1018)

2.2541654516726224


<h3>5. How many total orders did the top performing Brand have on Postmates?</h3>

In [25]:
# select data for Postmates
data_postmates = data[data['DeliveryPartner'] == 'postmates']

In [26]:
# check the brands on Postmates
print(data_postmates['Store'].unique())

["Chema's Chinese - Los Angeles" "Chema's Chinese - New York"
 "Gina's Grilled Cheese - Jacksonville" "Gina's Grilled Cheese - Miami"
 "Gina's Grilled Cheese - Orlando" "Gina's Grilled Cheese - Tampa"
 "Travis' Thai - Oakland"]


In [27]:
# create a new column listing only the store's brand without location
f = lambda x: x.split('-')[0]
data_postmates['Brand'] = data_postmates['Store'].apply(f)
print(data_postmates['Brand'].unique())

["Chema's Chinese " "Gina's Grilled Cheese " "Travis' Thai "]


C:\Users\titi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# calculate the total orders for each brand
total_orders = data_postmates.groupby('Brand')['OrderNumber'].count()
print(total_orders)

Brand
Chema's Chinese           159
Gina's Grilled Cheese      68
Travis' Thai               16
Name: OrderNumber, dtype: int64


<h3>6.What was the highest subtotal revenue for an Organization in Boston</h3>

In [29]:
# select data in Boston
data_temp = data_all.copy()
f = lambda x: x.split(' - ')[1]
data_temp['City'] = data_temp['Store'].apply(f)
data_boston = data_temp[data_temp['City'] == 'Boston']

In [30]:
# calculate subtotal for each brand
org_subtotal_boston = data_boston.groupby('org_name')['Subtotal'].sum()
print(org_subtotal_boston)

org_name
Acme Corp          606.50
Restaurant Co.    1210.05
Tasty Co.         8989.28
Name: Subtotal, dtype: float64


<h3>7. Which combination of organization and delivery platform had the lowest average delivery fee percentage?</h3>

In [30]:
# select data where delivery fee exist
data_delivery = data_temp[data_temp['OrderStatus'] == 'Delivered']

In [31]:
# calculate delivery fee percentage
data_delivery['deliveryFee_percent'] = data_delivery['Delivery Fee']/(data_delivery['Subtotal'] + data_delivery['Delivery Fee'])

C:\Users\titi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
# add the brand column to data_delivery table
f = lambda x: x.split(' - ')[0]
data_delivery['Brand'] = data_delivery['Store'].apply(f)

C:\Users\titi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
# find the mean value for every combination
delivery_fee_percent = data_delivery.groupby(['DeliveryPartner', 'Brand'])['deliveryFee_percent'].agg(['mean'])
print(delivery_fee_percent)

                                           mean
DeliveryPartner Brand                          
chownow         Gina's Grilled Cheese  0.079200
                Jen's Juice            0.074074
                Travis' Thai           0.056704
doordash        Chema's Chinese        0.080044
                Eli's Egg Sandos       0.070852
                Gina's Grilled Cheese  0.078373
                Jen's Juice            0.074097
                Travis' Thai           0.031922
                Trent's Tacos          0.061006
grubhub         Chema's Chinese        0.080001
                Eli's Egg Sandos       0.072472
                Jen's Juice            0.057453
                Travis' Thai           0.056712
grubhubweb      Chema's Chinese        0.070022
                Eli's Egg Sandos       0.072728
                Gina's Grilled Cheese  0.070564
                Jen's Juice            0.063493
                Travis' Thai           0.056693
postmates       Chema's Chinese        0

In [34]:
# find the combination with the lowest mean
delivery_fee_percent[delivery_fee_percent['mean'] == delivery_fee_percent['mean'].min()]

,,mean
DeliveryPartner,Brand,
doordash,Travis' Thai,0.031922


<h3>8. Which city had the highest cancellation rate for the week of 10/25-10/31</h3>

In [35]:
# transform data_temp table to have a column of order date without time
data_temp.loc[:,'orderDate'] = pd.to_datetime(data_temp.loc[:,'orderDate'])
data_temp['Date'] = data_temp.orderDate.dt.date

In [36]:
# select date between the week of 10/25-10/31 from data_temp table
date_1025 = datetime.strptime('2020-10-25', '%Y-%m-%d').date()
date_1031 = datetime.strptime('2020-10-31', '%Y-%m-%d').date()
condition_1025 = (data_temp['Date'] >= date_1025) & (data_temp['Date'] <= date_1031)
data_1025 = data_temp[condition_1025]

In [37]:
# print a list of cities
cities = data_1025.City.unique()
print(cities)

['Los Angeles' 'New York' 'Boston' 'Chicago' 'Philadelphia' 'Jacksonville'
 'Miami' 'Orlando' 'Tampa' 'Denver' 'Oakland' 'San Francisco' 'Austin'
 'Phoenix' 'San Diego']


In [38]:
# find out which city had the highest cancellation rate
num = 0
for city in cities:
    data_city = data_1025[data_1025['City'] == city]
    city_orderCount = data_city['OrderNumber'].count()
    city_cancelCount = data_city[data_city['OrderStatus'] == 'Canceled']['OrderNumber'].count()
    city_cancel_rate = city_cancelCount / city_orderCount
    if num < city_cancel_rate:
        print('new highest cancel rate is:', city, city_cancel_rate)
        num += city_cancel_rate
    else:
        pass

new highest cancel rate is: Los Angeles 0.02195121951219512
new highest cancel rate is: New York 0.022197558268590455
new highest cancel rate is: Boston 0.045454545454545456
new highest cancel rate is: San Francisco 0.10714285714285714


<h3>9. Which store had the highest % of their orders coming in the late-night daypart (10pm-4am)?</h3>

In [39]:
# create a binary column that indicate if the order comes in late-night
data_temp['Time'] = data_temp['orderDate'].dt.time
time_10pm = datetime.strptime('22:00:00', '%H:%M:%S').time()
time_4am  = datetime.strptime('04:00:00', '%H:%M:%S').time()
data_temp['late_night'] = np.where((data_temp['Time'] >= time_10pm) | (data_temp['Time'] <= time_4am), 1, 0)
# data_temp.head()

,StoreID,Store,OrderNumber,Subtotal,Delivery Fee,Total,DeliveryPartner,OrderStatus,orderDate,City,Date,Time,late_night
0,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04c9b627-8263-45f9-9219-d67cf56d0a13,12.99,1.13,NaN,postmates,Delivered,2020-10-26 17:56:05,Los Angeles,2020-10-26,17:56:05,0
1,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04fbb2d8-ae72-4ebb-b808-59ccebb839fe,24.47,2.13,NaN,doordash,Delivered,2020-10-23 01:23:55,Los Angeles,2020-10-23,01:23:55,1
2,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,05fe492e-33e1-4360-99fc-427c9f19fc8c,15.46,1.35,NaN,doordash,Picked Up,2020-10-28 20:08:20,Los Angeles,2020-10-28,20:08:20,0
3,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,09c0a9a0-a647-42df-9a70-1eb4069292ee,29.95,2.61,NaN,doordash,Delivered,2020-10-27 01:51:59,Los Angeles,2020-10-27,01:51:59,1
4,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0c1213dc-82c2-4625-aee0-816bc84a49d2,35.97,3.13,NaN,doordash,Delivered,2020-10-23 02:08:55,Los Angeles,2020-10-23,02:08:55,1
5,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0ef6e87f-8a28-4d34-9631-f51549c18662,51.92,4.52,NaN,doordash,Delivered,2020-10-28 15:12:18,Los Angeles,2020-10-28,15:12:18,0
6,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,132385aa-aaca-4faf-8344-067e26a791df,27.98,2.43,NaN,doordash,Delivered,2020-10-24 00:20:09,Los Angeles,2020-10-24,00:20:09,1
7,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,161003ff-c0a9-4054-aadf-8d5dd11091aa,17.98,1.56,NaN,doordash,Delivered,2020-10-23 22:33:07,Los Angeles,2020-10-23,22:33:07,1
8,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,1c6e9c67-6af2-4b36-960c-9bf5ee858005,26.97,2.35,NaN,doordash,Delivered,2020-10-28 14:07:18,Los Angeles,2020-10-28,14:07:18,0
9,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,22f87dcc-96fb-47d6-b942-799398156f16,15.98,1.39,NaN,ubereats,Delivered,2020-10-15 01:23:49,Los Angeles,2020-10-15,01:23:49,1


In [40]:
# find the latenight order number and total order number and calculate percentage
latenight_number = data_temp.groupby('Store')['late_night'].agg(['sum']) 
totalOrder_number = data_temp.groupby('Store')['OrderNumber'].agg(['count'])
new_table = pd.concat([latenight_number, totalOrder_number], axis = 1)
new_table['percent'] = new_table['sum']/new_table['count']
print(new_table)

                                      sum  count   percent
Store                                                     
Chema's Chinese - Los Angeles          19     49  0.387755
Chema's Chinese - New York            301   1087  0.276909
Eli's Egg Sandos - Boston             160    261  0.613027
Eli's Egg Sandos - Chicago            172    252  0.682540
Eli's Egg Sandos - Los Angeles        387    566  0.683746
Eli's Egg Sandos - New York           237    374  0.633690
Eli's Egg Sandos - Philadelphia       169    256  0.660156
Gina's Grilled Cheese - Jacksonville  230    313  0.734824
Gina's Grilled Cheese - Miami         122    158  0.772152
Gina's Grilled Cheese - Orlando       157    202  0.777228
Gina's Grilled Cheese - Tampa         147    189  0.777778
Jen's Juice - Boston                    2     60  0.033333
Jen's Juice - Denver                   33    172  0.191860
Travis' Thai - Boston                   0     18  0.000000
Travis' Thai - Los Angeles             69    270  0.2555

In [41]:
# find the highest late-night order store
print(new_table[new_table['percent'] == new_table['percent'].max()])

                               sum  count   percent
Store                                              
Gina's Grilled Cheese - Tampa  147    189  0.777778


<h3>10. Generic Question</h3>

    Assume Gina's Grilled Cheese wants to open a new city, which is open only for lunch (11am-3pm) on Saturday, Sunday, and Monday. At this location they would like to focus on maximizing their basket size rather than managing a large column of orders due to the limited staffing/space they will have available.
    
    Based on the date included, which city would likely be the best fit for Gina's Grilled Cheese to open in? Please explain your answer.

In [43]:
# add a brand column to data_temp table
f = lambda x: x.split(' - ')[0]
data_temp['Brand'] = data_temp['Store'].apply(f)
data_temp

,StoreID,Store,OrderNumber,Subtotal,Delivery Fee,Total,DeliveryPartner,OrderStatus,orderDate,City,Date,Time,late_night,Brand
0,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04c9b627-8263-45f9-9219-d67cf56d0a13,12.99,1.13,NaN,postmates,Delivered,2020-10-26 17:56:05,Los Angeles,2020-10-26,17:56:05,0,Chema's Chinese
1,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04fbb2d8-ae72-4ebb-b808-59ccebb839fe,24.47,2.13,NaN,doordash,Delivered,2020-10-23 01:23:55,Los Angeles,2020-10-23,01:23:55,1,Chema's Chinese
2,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,05fe492e-33e1-4360-99fc-427c9f19fc8c,15.46,1.35,NaN,doordash,Picked Up,2020-10-28 20:08:20,Los Angeles,2020-10-28,20:08:20,0,Chema's Chinese
3,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,09c0a9a0-a647-42df-9a70-1eb4069292ee,29.95,2.61,NaN,doordash,Delivered,2020-10-27 01:51:59,Los Angeles,2020-10-27,01:51:59,1,Chema's Chinese
4,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0c1213dc-82c2-4625-aee0-816bc84a49d2,35.97,3.13,NaN,doordash,Delivered,2020-10-23 02:08:55,Los Angeles,2020-10-23,02:08:55,1,Chema's Chinese
5,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0ef6e87f-8a28-4d34-9631-f51549c18662,51.92,4.52,NaN,doordash,Delivered,2020-10-28 15:12:18,Los Angeles,2020-10-28,15:12:18,0,Chema's Chinese
6,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,132385aa-aaca-4faf-8344-067e26a791df,27.98,2.43,NaN,doordash,Delivered,2020-10-24 00:20:09,Los Angeles,2020-10-24,00:20:09,1,Chema's Chinese
7,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,161003ff-c0a9-4054-aadf-8d5dd11091aa,17.98,1.56,NaN,doordash,Delivered,2020-10-23 22:33:07,Los Angeles,2020-10-23,22:33:07,1,Chema's Chinese
8,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,1c6e9c67-6af2-4b36-960c-9bf5ee858005,26.97,2.35,NaN,doordash,Delivered,2020-10-28 14:07:18,Los Angeles,2020-10-28,14:07:18,0,Chema's Chinese
9,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,22f87dcc-96fb-47d6-b942-799398156f16,15.98,1.39,NaN,ubereats,Delivered,2020-10-15 01:23:49,Los Angeles,2020-10-15,01:23:49,1,Chema's Chinese


In [47]:
# look at individual brands
brands = data_temp['Brand'].unique()
print(brands)

["Chema's Chinese" "Eli's Egg Sandos" "Gina's Grilled Cheese"
 "Jen's Juice" "Travis' Thai" "Trent's Tacos"]


In [48]:
# select data for brands other than Gina's Grilled Cheese since it wants to explore new locations
data_non_gina = data_temp[data_temp['Brand'] != 'Gina\'s Grilled Cheese']
# data_non_gina.head()

,StoreID,Store,OrderNumber,Subtotal,Delivery Fee,Total,DeliveryPartner,OrderStatus,orderDate,City,Date,Time,late_night,Brand
0,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04c9b627-8263-45f9-9219-d67cf56d0a13,12.99,1.13,NaN,postmates,Delivered,2020-10-26 17:56:05,Los Angeles,2020-10-26,17:56:05,0,Chema's Chinese
1,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,04fbb2d8-ae72-4ebb-b808-59ccebb839fe,24.47,2.13,NaN,doordash,Delivered,2020-10-23 01:23:55,Los Angeles,2020-10-23,01:23:55,1,Chema's Chinese
2,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,05fe492e-33e1-4360-99fc-427c9f19fc8c,15.46,1.35,NaN,doordash,Picked Up,2020-10-28 20:08:20,Los Angeles,2020-10-28,20:08:20,0,Chema's Chinese
3,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,09c0a9a0-a647-42df-9a70-1eb4069292ee,29.95,2.61,NaN,doordash,Delivered,2020-10-27 01:51:59,Los Angeles,2020-10-27,01:51:59,1,Chema's Chinese
4,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0c1213dc-82c2-4625-aee0-816bc84a49d2,35.97,3.13,NaN,doordash,Delivered,2020-10-23 02:08:55,Los Angeles,2020-10-23,02:08:55,1,Chema's Chinese
5,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,0ef6e87f-8a28-4d34-9631-f51549c18662,51.92,4.52,NaN,doordash,Delivered,2020-10-28 15:12:18,Los Angeles,2020-10-28,15:12:18,0,Chema's Chinese
6,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,132385aa-aaca-4faf-8344-067e26a791df,27.98,2.43,NaN,doordash,Delivered,2020-10-24 00:20:09,Los Angeles,2020-10-24,00:20:09,1,Chema's Chinese
7,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,161003ff-c0a9-4054-aadf-8d5dd11091aa,17.98,1.56,NaN,doordash,Delivered,2020-10-23 22:33:07,Los Angeles,2020-10-23,22:33:07,1,Chema's Chinese
8,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,1c6e9c67-6af2-4b36-960c-9bf5ee858005,26.97,2.35,NaN,doordash,Delivered,2020-10-28 14:07:18,Los Angeles,2020-10-28,14:07:18,0,Chema's Chinese
9,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,22f87dcc-96fb-47d6-b942-799398156f16,15.98,1.39,NaN,ubereats,Delivered,2020-10-15 01:23:49,Los Angeles,2020-10-15,01:23:49,1,Chema's Chinese


In [49]:
# select data for lunch (11am-3pm)
time_11am = datetime.strptime('11:00:00', '%H:%M:%S').time()
time_3pm  = datetime.strptime('15:00:00', '%H:%M:%S').time()
condition_lunch = (data_non_gina['Time'] >= time_11am) & (data_non_gina['Time'] <= time_3pm)
data_nongina_lunch = data_non_gina[condition_lunch]
# data_nongina_lunch.head()

,StoreID,Store,OrderNumber,Subtotal,Delivery Fee,Total,DeliveryPartner,OrderStatus,orderDate,City,Date,Time,late_night,Brand
8,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,1c6e9c67-6af2-4b36-960c-9bf5ee858005,26.97,2.35,NaN,doordash,Delivered,2020-10-28 14:07:18,Los Angeles,2020-10-28,14:07:18,0,Chema's Chinese
30,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,79f70202-92bb-4823-b23c-41cd60eeef1a,25.98,2.26,NaN,postmates,Delivered,2020-10-23 14:46:12,Los Angeles,2020-10-23,14:46:12,0,Chema's Chinese
42,b23cc045-e8b9-4bdd-9382-b942399c8c03,Chema's Chinese - Los Angeles,d85172ca-6b53-4ea1-9f49-542d2d99db2b,21.96,1.91,NaN,doordash,Delivered,2020-10-28 13:00:17,Los Angeles,2020-10-28,13:00:17,0,Chema's Chinese
82,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,067a3d88-44b1-475a-8cb5-2ffd0a4b389e,21.38,1.86,NaN,doordash,Delivered,2020-10-31 11:30:36,New York,2020-10-31,11:30:36,0,Chema's Chinese
86,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,07730d4f-57fa-4dc9-b4d2-e1da115892e6,19.10,1.66,NaN,doordash,Delivered,2020-10-26 12:32:33,New York,2020-10-26,12:32:33,0,Chema's Chinese
89,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,07e63d72-d5e3-418c-82e7-b4c51e6c42b6,10.74,0.93,NaN,doordash,Delivered,2020-10-30 11:49:22,New York,2020-10-30,11:49:22,0,Chema's Chinese
92,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,08771221-60a6-4117-85f1-f5637273f2b1,27.46,2.39,NaN,doordash,Delivered,2020-10-15 13:20:04,New York,2020-10-15,13:20:04,0,Chema's Chinese
97,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,0944a434-da95-4fd1-a3d1-94c77ece3e05,18.96,1.65,NaN,doordash,Delivered,2020-10-27 12:20:43,New York,2020-10-27,12:20:43,0,Chema's Chinese
98,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,09b74fe7-bed7-4ed7-b4b2-98fe681b1093,13.98,1.22,15.20,grubhubweb,Delivered,2020-10-25 14:13:19,New York,2020-10-25,14:13:19,0,Chema's Chinese
102,141b4012-1297-4364-9a59-f1d1efe3b01f,Chema's Chinese - New York,0abb4b54-e847-4994-a367-8065d2e736af,12.97,1.13,NaN,doordash,Delivered,2020-10-28 13:41:56,New York,2020-10-28,13:41:56,0,Chema's Chinese


In [60]:
# select data for sat, sun, mon
f = lambda x: x.weekday()
data_nongina_lunch['Weekday'] = data_nongina_lunch['Date'].apply(f)
data_nongina_lunchWeek = data_nongina_lunch[data_nongina_lunch['Weekday'].isin([5, 6, 0])]
# data_nongina_lunchWeek.head()

C:\Users\titi1\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
# check the median subtotal in each city at lunchtime
medSubtotal = data_nongina_lunchWeek.groupby(['City'])['Subtotal'].agg(['median','count'])
medSubtotal

,median,count
City,,
Austin,13.280,6
New York,16.975,26
Oakland,144.100,1
Phoenix,20.625,6


In [75]:
# check the median subtotal for each brand in each city at lunchtime
medSubtotal_brand = data_nongina_lunchWeek.groupby(['City','Brand'])['Subtotal'].agg(['median','count'])
medSubtotal_brand

,,median,count
City,Brand,,
Austin,Trent's Tacos,13.280,6
New York,Chema's Chinese,16.975,26
Oakland,Travis' Thai,144.100,1
Phoenix,Trent's Tacos,20.625,6


In [72]:
# check the median subtotal for each brand in each city for everyday lunchtime
medSubtotal2 = data_nongina_lunch.groupby(['City','Brand'])['Subtotal'].agg(['median','count'])
medSubtotal2

median  count
City        Brand                          
Austin      Trent's Tacos     17.570     11
Boston      Jen's Juice       13.950      1
Denver      Jen's Juice       44.000      1
Los Angeles Chema's Chinese   25.980      3
            Travis' Thai      13.125      6
New York    Chema's Chinese   16.480     73
            Eli's Egg Sandos  16.230      1
            Travis' Thai      23.725      4
Oakland     Travis' Thai      23.000      3
Phoenix     Trent's Tacos     21.970      9
San Diego   Trent's Tacos     20.980      5

In this research, I used the median subtotal to measure backet size, and calculated the average subtotal for each city at sat, sun, mon lunchtime. The data point is less than sufficient, so I went on to calculate the average subtotal for everyday lunchtime.

Among data present in both tables, it seems that customers in Phoenix has a more consistent pattern in making large orders in lunch time. Oakland is also high in backet size but we need more data to confirm. Therefore I would recommend Phoenix for now.

Additionally, we can see that our database doesn't have brands open at lunchtime in San Diego, Los Angeles, Denver, and Boston on sat, sun, mon yet, so it might be an opportunity to share the market size there. Among these Denver has the highest average subtotal, but I would recommend San Diego and Los Angeles since we have more data points, although we need further research since the lack of data point might indicate customer base there order less on sat, sun, mon lunchtime.